<a href="https://colab.research.google.com/github/fleshgordo/sensinglikeamultiplicity/blob/main/day2/Augmenting_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Augmenting data via Google Earth Engine

Following the initial study for the [blurred sites](https://en.m.wikipedia.org/wiki/List_of_satellite_map_images_with_missing_or_unclear_data) of Google Earth, this notebook expands on the topic of satellite images and remote sensing via Google Earth Engine. It introduces how to access Image layers from GEE, tweak their appearances to build webmaps, and run batch downloads for many locations at once to finally analyse relationship of similarity among the original sites via PCA and t-SNE. Python and the ee, geopandas, scikit-learn, PIL libraries will be used.

In [50]:
!pip install -q geopandas

import ee
import os
import pandas as pd
import geopandas as gpd
import requests
from tqdm import tqdm
from PIL import Image, ImageFilter

In order to access GEE, we first need to authenticate.
Follow the instructions to obtain a valid key, and paste it back in the prompt.

Remember to first [sign up](https://earthengine.google.com/signup) to GEE to use the service.

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

If the above step fails to run, make sure that the email you are providing matches the email that you used to sign up

## Explore the database

Let's check the connection to GEE following the official [documentation](https://developers.google.com/earth-engine/guides/python_install-colab).

In [52]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


As Google Earth Engine's primary scope is to support web-based visualizations, it is possible to explore GEE interactively via the Folium python library. 
*Folium makes it easy to visualize data that’s been manipulated in Python on an interactive leaflet map.*

[Folium official documentation](https://python-visualization.github.io/folium/)

In [ ]:
import folium

### Basemaps

In [75]:
#Explore basemaps
# https://leaflet-extras.github.io/leaflet-providers/preview/

folium.Map(location=[46.519962,6.633597], 
    # tiles='stamentoner'
    # tiles='stamenterrain'
    # tiles='stamenwatercolor'
    # tiles='cartoDBpositron'
    tiles='cartoDBdark_matter'
)

### GEE tiles
In order to import Google Earth Engine data on our Folium web map, we define a function to fetch tiles from GEE and we overlay it onto the Map

In [90]:
def add_ee_layer(self, image, vis_params, name):
  map_id_dict = image.getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [120]:
# Create a basemap
ee_map = folium.Map(location=[46.519962,6.633597], tiles='cartoDBdark_matter')

# Set visualization parameters
layer = ee.Image('USGS/SRTMGL1_003').select("elevation")

# Filter the values
layer = layer.updateMask(layer.gt(250))

# Apply style
vis_params = {
  'min': 0,
  'max': 2000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Add the layer to the map object
ee_map.add_ee_layer(layer, vis_params, 'DEM')

# Add the layer control panel and display the map
ee_map.add_child(folium.LayerControl())
display(ee_map)

Change the layer and the bands instructions to load a different image. Make sure to apply a min & max pair that is meaningful to the data that is being shown

In [118]:
# Create a basemap
ee_map = folium.Map(location=[46.519962,6.633597], tiles='cartoDBpositron')

# Set visualization parameters
layer = ee.Image('UMD/hansen/global_forest_change_2021_v1_9').select("treecover2000")

# Filter the values
layer = layer.updateMask(layer.gt(1))

vis_params = {
  'min': 0,
  'max': 100,
  'palette': ['white','salmon','green']}

# Add the layer to the map object
ee_map.add_ee_layer(layer, vis_params, 'layer')

# Add the layer control panel and display the map
ee_map.add_child(folium.LayerControl())
display(ee_map)

Masks can also come from other layers or bands

In [116]:
# Create a basemap
ee_map = folium.Map(location=[46.519962,6.633597], tiles='cartoDBpositron')

# Set visualization parameters
layer = ee.Image('UMD/hansen/global_forest_change_2021_v1_9').select("treecover2000")
mask = ee.Image('UMD/hansen/global_forest_change_2021_v1_9').select("loss")

# Filter the values
layer = layer.updateMask(layer.gt(1)).updateMask(mask.gt(0))

vis_params = {
  'min': 0,
  'max': 100,
  'palette': ['white','salmon','red']}

# Add the layer to the map object
ee_map.add_ee_layer(layer, vis_params, 'layer')

# Add the layer control panel and display the map
ee_map.add_child(folium.LayerControl())
display(ee_map)

Finally let's download an image for a specific area of interest using the *ee.Image.getThumbUrl()* function

In [126]:
# Import the Image function from the IPython.display module.
from IPython.display import Image

# check the https://boundingbox.klokantech.com/ to get the area of analysis
region = ee.Geometry.Rectangle([5.9132,46.0068,7.2535,46.898])
layer = ee.Image('USGS/SRTMGL1_003').select("elevation")
vis_params = {
  'min': 1,
  'max': 2000,
  'palette': ['white', 'black'],
  'res': 300}

# Display a thumbnail of global elevation.
url = layer.getThumbURL({'min': vis_params["min"], 
                         'max': vis_params["max"], 
                         'dimensions': vis_params["res"],
                         'region':region, 
                         'crs': 'EPSG:3857',
                         'palette': vis_params["palette"]})
url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/4bdc0103657a5b9f9a03f370508eaf53-fb45dd9e3c7cb1eeb91abb71bd76e9de:getPixels'

In [128]:
Image(url=url)

## Batch download


In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Define a harvesting function
In order to download images from multiple locations we create a function that loops through an array of locations and stores in a GDrive folder the outputs from an ee.Image layer.

More precisely, the *download_img()* function takes as inputs:


*   the **image** layer description alongside its specific **band**
*   the geographical bounding box (**bbox**) and the resolution (**res**) in pixel of the final output
*   the color **palette** to apply and the min & max **bounds** to stretch the colors
*   finally the output subfolder (**sub_dir**) to store the images

In [99]:
# Select the output folder
def download_img(image, band, bbox, vis_param, sub_dir) :

    # General settings 
    file_type = "jpg"    
    out_dir = "/content/gdrive/MyDrive/GEE/" # the outputs will be stored in out_dir/sub_dir

    # Prepare the folder to export the images
    try:
        os.mkdir(os.path.join(out_dir))
        print("main_dir created")
    except:
        print("main_dir already existing")
    try:
        os.mkdir(os.path.join(out_dir,sub_dir))
        print("sub_dir created")
    except:
        print("sub_dir already existing")

    # Launch the loop
    for i in tqdm(range(0, bbox.shape[0])):

        region = ee.Geometry.Rectangle(bbox.iloc[i]['minx'],bbox.iloc[i]['miny'],bbox.iloc[i]['maxx'],bbox.iloc[i]['maxy'])
        url = image.getThumbURL({ 
                            'bands': band,
                            'min': vis_param["min"],
                            'max': vis_param["max"],
                            'palette': vis_param["palette"],
                            'dimensions': vis_param["res"],
                            'crs': 'EPSG:3857',
                            'region': region,
                            'format': file_type
                        })
                        
        file_name = ("image_" + str(bbox.iloc[i]['id']) + "."+file_type)
        out_path = os.path.join(out_dir, sub_dir, file_name)

        # Save the image for the generated url in the specified path
        response = requests.get(url)
        open(out_path, "wb").write(response.content)

### Import locations

we read a .csv with location data with pandas that after we convert into georeferenced geometries in order to calculate bounding boxes (with a radius in m) and download pictures.

In [25]:
url = "https://raw.githubusercontent.com/fleshgordo/sensinglikeamultiplicity/main/day2/data/blurred_sites.csv"
df = pd.read_csv(url, delimiter=",", header=0)

we convert the pandas dataframe into a collection of point geometries with the geopandas library and using the values stored in the "lon" and "lat" columns. The coordinates are in WSG84 (Lon Lat) therefore we use as crs the default settings (*EPSG:4326*)

In [26]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['lon'], df['lat']), crs = 'EPSG:4326')

As the locations are **point-based data**, we create geographical regions of interest to later extract images using a radius in meters. This is done by first reprojecting the geometries in a meter-united crs (*EPSG:3857*) and then by making a **buffer** to extract the mininum and maximum extensions. Finally we convert the geometries again to *4326* to facilitate the communication with  GEE.

In [27]:
radius_m = 10000

buff = gdf.to_crs('EPSG:3857').buffer(radius_m).to_crs('EPSG:4326')
bbox = gdf.join(buff.envelope.bounds)

### Run the loop


In [100]:
#  Compute the Built-up
image = ee.Image('JRC/GHSL/P2016/BUILT_LDSMT_GLOBE_V1');
vis_params = {
  'min': 1,
  'max': 6,
  'palette': ['white', 'black'],
  'res': 300}
download_img (image, "built", bbox,  vis_params, "built")

main_dir already existing
sub_dir already existing


100%|██████████| 98/98 [01:23<00:00,  1.18it/s]
